In [1]:
import requests
import urllib.error
import urllib.parse
from urllib import request
import json
import pandas as pd
import numpy as np
import re
import time

In [2]:
def read_the_url(urllink):
    res = requests.get(urllink)
    js = json.loads(res.text)
    return js


def test_token(js):
    try:
        token = js['next_page_token']
    except KeyError:
        return True ### there is no token
    else:
        return False ### there is a token
    
def get_latitude_longtitude(address):
    # decode url
    address = urllib.parse.quote(address)
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=" + address + "&key=" #add your key
    
    while True:
        res = requests.get(url)
        js = json.loads(res.text)

        if js["status"] != "OVER_QUERY_LIMIT":
            time.sleep(1)
            break

    result = js["results"][0]["geometry"]["location"]
    lat = result["lat"]
    lng = result["lng"]

    return lat, lng
    
    
def get_the_place_id(url):
    list_of_name=[]
    place_id_list=[]
    js = read_the_url(url)
    
    for i in range(1,4):
        if test_token(js) is True:
            for item in js['results']:
                list_of_name.append(item['name'])
                place_id_list.append(item['place_id'])
            break
        else:
            token = js['next_page_token']
            URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken=" + token + "&key=" #add your key
            for item in js['results']:
                list_of_name.append(item['name'])
                place_id_list.append(item['place_id'])
            time.sleep(2)
            js = read_the_url(URL)
        
    df = pd.DataFrame(list_of_name, columns=['name'])
    df = pd.concat([df, pd.DataFrame(place_id_list,columns=['place_id'])],axis=1)
    
    return df


def get_street_address_and_suburb(detail):
    street_address = ''
    suburb_info = ''
    street_number = ''
    route = ''
    suburb = ''
    for item in detail['result']['address_components']:
        if item['types'] == ['street_number']:
            street_number = item['long_name']
        
        elif item['types'] == ['route']:
            route = item['long_name']
            
        elif item['types'] == ['locality', 'political']:
            suburb = item['long_name']
            
    street_address = street_number + ' '+ route
    suburb_info = suburb
    
    return street_address, suburb_info

In [3]:
dataframe = pd.read_csv('op_shop_list.csv')

In [4]:
for i in range(len(dataframe)):
    lat = str(dataframe['lat'][i])
    lng = str(dataframe['lng'][i])
    URL_of_opshop = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="+lat+","+lng+"&radius=50&keyword=op+shop&key=" #add your key
    if i == 0:
        op_shop_df = get_the_place_id(URL_of_opshop)
    
    else:
        temper_df = get_the_place_id(URL_of_opshop)
        op_shop_df = op_shop_df.append(temper_df)
        op_shop_df.drop_duplicates(inplace=True)

In [5]:
op_shop_df.reset_index(drop=True,inplace=True)

In [6]:
op_shop_df = op_shop_df.drop(op_shop_df.index[801:len(op_shop_df)])

In [7]:
op_shop_df['address'] = ''
op_shop_df['street_address'] = ''
op_shop_df['suburb'] = ''
op_shop_df['lat'] = ''
op_shop_df['lng'] = ''
op_shop_df['phone_number'] = ''
op_shop_df['rating'] = ''
op_shop_df['Monday'] = ''
op_shop_df['Tuesday'] = ''
op_shop_df['Wednesday'] = ''
op_shop_df['Thursday'] = ''
op_shop_df['Friday'] = ''
op_shop_df['Saturday'] = ''
op_shop_df['Sunday'] = ''

In [9]:
op_shop_df.to_csv('op shop list only id.csv')

In [11]:
for i in range(len(op_shop_df)):
    place_id = op_shop_df['place_id'][i]
    URL_of_place_id = "https://maps.googleapis.com/maps/api/place/details/json?place_id=" + place_id + "&key=" #add your key
    detail = read_the_url(URL_of_place_id)
    op_shop_df['address'][i] = detail['result']['formatted_address']
    lat, lng = get_latitude_longtitude(detail['result']['formatted_address'])
    op_shop_df['lat'][i] = lat
    op_shop_df['lng'][i] = lng
    street, suburb = get_street_address_and_suburb(detail)
     
    try:
        op_shop_df['phone_number'][i] =  detail['result']['formatted_phone_number']
    except KeyError:
        op_shop_df['phone_number'][i] = 'unknown'
    else:
        op_shop_df['phone_number'][i] = detail['result']['formatted_phone_number']
        
    try:
        op_shop_df['rating'][i] =  detail['result']['rating']
    except KeyError:
        op_shop_df['rating'][i] = 'no rating'
    else:
        op_shop_df['rating'][i] = detail['result']['rating']
        
    try:
        detail['result']['opening_hours']['weekday_text']
    except KeyError:
        op_shop_df['Monday'][i] = 'please call for detail'
        op_shop_df['Tuesday'][i] = 'please call for detail'
        op_shop_df['Wednesday'][i] = 'please call for detail'
        op_shop_df['Thursday'][i] = 'please call for detail'
        op_shop_df['Friday'][i] = 'please call for detail'
        op_shop_df['Saturday'][i] = 'please call for detail'
        op_shop_df['Sunday'][i] = 'please call for detail'
    else:
        op_shop_df['Monday'][i] = detail['result']['opening_hours']['weekday_text'][0]
        op_shop_df['Tuesday'][i] = detail['result']['opening_hours']['weekday_text'][1]
        op_shop_df['Wednesday'][i] = detail['result']['opening_hours']['weekday_text'][2]
        op_shop_df['Thursday'][i] = detail['result']['opening_hours']['weekday_text'][3]
        op_shop_df['Friday'][i] = detail['result']['opening_hours']['weekday_text'][4]
        op_shop_df['Saturday'][i] = detail['result']['opening_hours']['weekday_text'][5]
        op_shop_df['Sunday'][i] = detail['result']['opening_hours']['weekday_text'][6]
    
    op_shop_df['street_address'][i] = street
    op_shop_df['suburb'][i] = suburb

In [12]:
op_shop_df.head()

,name,place_id,address,street_address,suburb,lat,lng,phone_number,rating,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Salvos Stores Bourke St,ChIJFfjEmshC1moR5rKP3V2LMCQ,"69 Bourke St, Melbourne VIC 3000, Australia",69 Bourke Street,Melbourne,-37.8121,144.971,(03) 9663 7615,4.1,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail
1,Hunter Gatherer,ChIJ1V2CjmZo1moROkpP_5qsrtU,"335 Bourke St, Melbourne VIC 3000, Australia",335 Bourke Street,Melbourne,-37.8145,144.964,(03) 9650 1843,3.8,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail
2,The Conscious Closet,ChIJYZrPz8tC1moRrNwbbtGLhOU,"210 Lonsdale St, Melbourne VIC 3000, Australia",210 Lonsdale Street,Melbourne,-37.8109,144.966,(03) 9662 4289,3.6,Monday: Closed,Tuesday: 12:00 – 5:00 PM,Wednesday: 12:00 – 5:00 PM,Thursday: 12:00 – 5:00 PM,Friday: 12:00 – 5:00 PM,Saturday: 12:00 – 5:00 PM,Sunday: Closed
3,Salvos Stores Footscray,ChIJBzKFgZJd1moRGys-QeQy6vQ,"199 Barkly St, Footscray VIC 3011, Australia",199 Barkly Street,Footscray,-37.7997,144.898,(03) 9689 7811,3.9,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail
4,Savers,ChIJxRfcqY1d1moRP0D3sMqxJO8,"33 Albert St, Footscray VIC 3011, Australia",33 Albert Street,Footscray,-37.8017,144.898,(03) 9689 6811,4.2,Monday: 10:00 AM – 5:00 PM,Tuesday: 10:00 AM – 5:00 PM,Wednesday: 10:00 AM – 5:00 PM,Thursday: 10:00 AM – 5:00 PM,Friday: 10:00 AM – 5:00 PM,Saturday: 10:00 AM – 5:00 PM,Sunday: 11:00 AM – 5:00 PM


In [ ]:
op_shop_df.to_csv('temp_data_of_op_shop.csv')

In [13]:
op_shop_df.drop(columns=['place_id'], inplace=True)

In [3]:
df = pd.read_csv('temp_data_of_op_shop.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [6]:
df.head()

,Unnamed: 0,name,address,street_address,suburb,lat,lng,phone_number,rating,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,Salvos Stores Bourke St,"69 Bourke St, Melbourne VIC 3000, Australia",69 Bourke Street,Melbourne,-37.812081,144.971416,(03) 9663 7615,4.1,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail
1,1,Hunter Gatherer,"335 Bourke St, Melbourne VIC 3000, Australia",335 Bourke Street,Melbourne,-37.814462,144.964064,(03) 9650 1843,3.8,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail
2,2,The Conscious Closet,"210 Lonsdale St, Melbourne VIC 3000, Australia",210 Lonsdale Street,Melbourne,-37.810882,144.965766,(03) 9662 4289,3.6,Closed,12:00 – 5:00 PM,12:00 – 5:00 PM,12:00 – 5:00 PM,12:00 – 5:00 PM,12:00 – 5:00 PM,Closed
3,3,Salvos Stores Footscray,"199 Barkly St, Footscray VIC 3011, Australia",199 Barkly Street,Footscray,-37.799682,144.897569,(03) 9689 7811,3.9,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail
4,4,Savers,"33 Albert St, Footscray VIC 3011, Australia",33 Albert Street,Footscray,-37.801721,144.898025,(03) 9689 6811,4.2,10:00 AM – 5:00 PM,10:00 AM – 5:00 PM,10:00 AM – 5:00 PM,10:00 AM – 5:00 PM,10:00 AM – 5:00 PM,10:00 AM – 5:00 PM,11:00 AM – 5:00 PM


In [15]:
op_shop_df['open_time'] = ''

In [ ]:
for i in range(len(op_shop_df)):
    list_of_dict = []
    monday = op_shop_df['Monday'][i]
    tuesday = op_shop_df['Tuesday'][i]
    wednesday = op_shop_df['Wednesday'][i]
    thursday = op_shop_df['Thursday'][i]
    friday = op_shop_df['Friday'][i]
    saturday = op_shop_df['Saturday'][i]
    sunday = op_shop_df['Sunday'][i]

    dict_of_time = {'Monday' : monday}
    list_of_dict.append(dict_of_time)
    dict_of_time = {'Tuesday' : tuesday}
    list_of_dict.append(dict_of_time)
    dict_of_time = {'Wednesday' : wednesday}
    list_of_dict.append(dict_of_time)
    dict_of_time = {'Thursday' : thursday}
    list_of_dict.append(dict_of_time)
    dict_of_time = {'Friday' : friday}
    list_of_dict.append(dict_of_time)m
    dict_of_time = {'Saturday' : saturday}
    list_of_dict.append(dict_of_time)
    dict_of_time = {'Sunday' : sunday}
    list_of_dict.append(dict_of_time)
    
    op_shop_df['open_time'][i] = list_of_dict

In [37]:
op_shop_df

,name,address,street_address,suburb,lat,lng,phone_number,rating,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,open_time
0,Salvos Stores Bourke St,"69 Bourke St, Melbourne VIC 3000, Australia",69 Bourke Street,Melbourne,-37.8121,144.971,(03) 9663 7615,4.1,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,"[{'Monday': 'please call for detail'}, {'Tuesd..."
1,Hunter Gatherer,"335 Bourke St, Melbourne VIC 3000, Australia",335 Bourke Street,Melbourne,-37.8145,144.964,(03) 9650 1843,3.8,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,"[{'Monday': 'please call for detail'}, {'Tuesd..."
2,The Conscious Closet,"210 Lonsdale St, Melbourne VIC 3000, Australia",210 Lonsdale Street,Melbourne,-37.8109,144.966,(03) 9662 4289,3.6,Monday: Closed,Tuesday: 12:00 – 5:00 PM,Wednesday: 12:00 – 5:00 PM,Thursday: 12:00 – 5:00 PM,Friday: 12:00 – 5:00 PM,Saturday: 12:00 – 5:00 PM,Sunday: Closed,"[{'Monday': ' Closed'}, {'Tuesday': ' 12:00 – ..."
3,Salvos Stores Footscray,"199 Barkly St, Footscray VIC 3011, Australia",199 Barkly Street,Footscray,-37.7997,144.898,(03) 9689 7811,3.9,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,please call for detail,"[{'Monday': 'please call for detail'}, {'Tuesd..."
4,Savers,"33 Albert St, Footscray VIC 3011, Australia",33 Albert Street,Footscray,-37.8017,144.898,(03) 9689 6811,4.2,Monday: 10:00 AM – 5:00 PM,Tuesday: 10:00 AM – 5:00 PM,Wednesday: 10:00 AM – 5:00 PM,Thursday: 10:00 AM – 5:00 PM,Friday: 10:00 AM – 5:00 PM,Saturday: 10:00 AM – 5:00 PM,Sunday: 11:00 AM – 5:00 PM,"[{'Monday': ' 10:00 AM – 5:00 PM'}, {'Tuesday'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,Dimmeys Mildura,"135 Ninth St, Mildura VIC 3500, Australia",135 Ninth Street,Mildura,-34.186,142.16,(03) 5021 5500,3.9,Monday: 9:30 AM – 4:30 PM,Tuesday: 9:30 AM – 4:30 PM,Wednesday: 9:30 AM – 4:30 PM,Thursday: 9:30 AM – 4:30 PM,Friday: 9:30 AM – 4:30 PM,Saturday: 9:30 AM – 1:00 PM,Sunday: Closed,"[{'Monday': ' 9:30 AM – 4:30 PM'}, {'Tuesday':..."
797,Bridgestone Service Centre - Mildura City Heart,"155 Ninth St, Mildura VIC 3500, Australia",155 Ninth Street,Mildura,-34.1844,142.158,(03) 5023 0381,4.6,Monday: 8:00 AM – 5:00 PM,Tuesday: 8:00 AM – 5:00 PM,Wednesday: 8:00 AM – 5:00 PM,Thursday: 8:00 AM – 5:00 PM,Friday: 8:00 AM – 5:00 PM,Saturday: Closed,Sunday: Closed,"[{'Monday': ' 8:00 AM – 5:00 PM'}, {'Tuesday':..."
798,My Chemist,"69 Langtree Ave, Mildura VIC 3500, Australia",69 Langtree Avenue,Mildura,-34.1848,142.16,(03) 5023 1844,3.6,Monday: 9:00 AM – 5:30 PM,Tuesday: 9:00 AM – 5:30 PM,Wednesday: 9:00 AM – 5:30 PM,Thursday: 9:00 AM – 5:30 PM,Friday: 9:00 AM – 6:00 PM,Saturday: 9:00 AM – 3:00 PM,Sunday: Closed,"[{'Monday': ' 9:00 AM – 5:30 PM'}, {'Tuesday':..."
799,Anglicare Mildura,"138 Pine Ave, Mildura VIC 3500, Australia",138 Pine Avenue,Mildura,-34.1861,142.156,(03) 5025 9300,5,Monday: 9:00 AM – 5:00 PM,Tuesday: 9:00 AM – 5:00 PM,Wednesday: 9:00 AM – 5:00 PM,Thursday: 9:00 AM – 5:00 PM,Friday: 9:00 AM – 5:00 PM,Saturday: Closed,Sunday: Closed,"[{'Monday': ' 9:00 AM – 5:00 PM'}, {'Tuesday':..."


In [38]:
jsonlist = []
for i in range(len(op_shop_df)):
    aItem = {}
    aItem["id"] = i
    aItem["name"] = op_shop_df['name'][i]
    aItem["address"] = op_shop_df['address'][i]
    aItem['lat'] = op_shop_df['lat'][i]
    aItem['lng'] = op_shop_df['lng'][i]
    aItem['street'] = op_shop_df['street_address'][i]
    aItem['suburb'] = op_shop_df['suburb'][i]
    aItem['open_time'] = op_shop_df['open_time'][i]
    aJson = json.dumps(aItem)
    jsonlist.append(aItem)

In [39]:
savefile = 'final op shop list.json'

with open(savefile, 'w') as f:
    f.write(json.dumps(jsonlist))